In [28]:
import re
import sys

## 标记数据处理

In [68]:
file = "biaoji.txt"

all_ner_data = []
with open(file, encoding="utf-8") as f:
    for s in f.readlines():
        s = s.strip('\n')
        ner_data = []
        result_1 = re.finditer(r'\[\@', s)
        result_2 = re.finditer(r'\*\]', s)
        begin = []
        end = []
        for each in result_1:
            begin.append(each.start())
        for each in result_2:
            end.append(each.end())
        assert len(begin) == len(end)
        i = 0
        j = 0
        while i < len(s):
            if i not in begin:
                ner_data.append([s[i], 'O'])
                i = i + 1
            else:
                ann = s[i + 2:end[j] - 2]
                entity, ner = ann.rsplit('#')
                if (len(entity) == 1):
                    ner_data.append([entity, 'S-' + ner])
                else:
                    if (len(entity) == 2):
                        ner_data.append([entity[0], 'B-' + ner])
                        ner_data.append([entity[1], 'E-' + ner])
                    else:
                        ner_data.append([entity[0], 'B-' + ner])
                        for n in range(1, len(entity) - 1):
                            ner_data.append([entity[n], 'I-' + ner])
                        ner_data.append([entity[-1], 'E-' + ner])
        
                i = end[j]
                j = j + 1
        all_ner_data.append(ner_data)
f.close()

In [54]:
all_ner_data[0]

[['上', 'B-Location'],
 ['海', 'E-Location'],
 ['今', 'O'],
 ['天', 'O'],
 ['的', 'O'],
 ['天', 'O'],
 ['气', 'O'],
 ['怎', 'O'],
 ['么', 'O'],
 ['样', 'O']]

In [55]:
all_ner_data_list = []
for seq_list in all_ner_data:
    zi = []
    mark = []
    for zi_mark in seq_list:
        zi.append(zi_mark[0])
        mark.append(zi_mark[1])
        seq_tuple = (zi, mark)
    all_ner_data_list.append(seq_tuple)

In [56]:
all_ner_data_list

[(['上', '海', '今', '天', '的', '天', '气', '怎', '么', '样'],
  ['B-Location', 'E-Location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']),
 (['杭', '州', '今', '天', '有', '点', '下', '雨'],
  ['B-Location', 'E-Location', 'O', 'O', 'O', 'O', 'O', 'O']),
 (['昨', '天', '成', '都', '出', '太', '阳', '了'],
  ['O', 'O', 'B-Location', 'E-Location', 'O', 'O', 'O', 'O']),
 (['今', '年', '北', '京', '的', '空', '气', '不', '太', '好'],
  ['O', 'O', 'B-Location', 'E-Location', 'O', 'O', 'O', 'O', 'O', 'O']),
 (['冬', '天', '的', '哈', '尔', '滨', '冰', '天', '雪', '地'],
  ['O',
   'O',
   'O',
   'B-Location',
   'I-Location',
   'E-Location',
   'O',
   'O',
   'O',
   'O']),
 (['武', '汉', '有', '很', '多', '樱', '花', '树'],
  ['B-Location', 'E-Location', 'O', 'O', 'O', 'O', 'O', 'O']),
 (['金', '华', '生', '产', '的', '火', '腿', '很', '出', '名'],
  ['B-Location', 'E-Location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']),
 (['上', '海', '在', '地', '图', '上', '紧', '挨', '着', '杭', '州'],
  ['B-Location',
   'E-Location',
   'O',
   'O',
   'O',
   'O',
   'O',
  

## 训练

In [57]:
"""
pytorch实现BiLSTM+CRF用于NER(命名实体识别)
https://blog.csdn.net/zycxnanwang/article/details/90385259
https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html#bi-lstm-conditional-random-field-discussion
"""

import torch
import torch.nn as nn
import torch.optim as optim
# START_TAG = "<START>"
# STOP_TAG = "<STOP>"
# EMBEDDING_DIM = 5
# HIDDEN_DIM = 4
# torch.manual_seed(1)

def prepare_sequence(seq, to_ix):
    """句子转index"""
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        
        print("embedding_dim", embedding_dim)
        print("hidden_dim", hidden_dim)
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        # 转移概率矩阵
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        """初始隐藏状态"""
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full([self.tagset_size], -10000.)
        # START_TAG has all of the score.
        init_alphas[self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        # Iterate through the sentence
        forward_var_list=[]
        forward_var_list.append(init_alphas)
        for feat_index in range(feats.shape[0]):
            gamar_r_l = torch.stack([forward_var_list[feat_index]] * feats.shape[1])
            t_r1_k = torch.unsqueeze(feats[feat_index],0).transpose(0,1)
            aa = gamar_r_l + t_r1_k + self.transitions
            forward_var_list.append(torch.logsumexp(aa,dim=1))
        terminal_var = forward_var_list[-1] + self.transitions[self.tag_to_ix[STOP_TAG]]
        terminal_var = torch.unsqueeze(terminal_var,0)
        alpha = torch.logsumexp(terminal_var, dim=1)[0]
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        # print("embeds", embeds)
        # 打印了embeds，确实每个字和EMBEDDING_DIM匹配
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        # print("lstm_out:", lstm_out)
        lstm_feats = self.hidden2tag(lstm_out)  # 过一个线形层
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        """维特比算法"""
        backpointers = []
        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var_list = []
        forward_var_list.append(init_vvars)

        for feat_index in range(feats.shape[0]):
            gamar_r_l = torch.stack([forward_var_list[feat_index]] * feats.shape[1])
            gamar_r_l = torch.squeeze(gamar_r_l)
            next_tag_var = gamar_r_l + self.transitions
            viterbivars_t,bptrs_t = torch.max(next_tag_var,dim=1)

            t_r1_k = torch.unsqueeze(feats[feat_index], 0)
            forward_var_new = torch.unsqueeze(viterbivars_t,0) + t_r1_k

            forward_var_list.append(forward_var_new)
            backpointers.append(bptrs_t.tolist())

        # Transition to STOP_TAG
        terminal_var = forward_var_list[-1] + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = torch.argmax(terminal_var).tolist()
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path


    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  
        """
        输出，每个字的tag概率
        
        tag包括了BIO，以及<START><STOP>所以是5维
        """
        # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)
        # print(lstm_feats)
        # print(lstm_feats.shape)
        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [58]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 20
HIDDEN_DIM = 64

training_data = all_ner_data_list

In [59]:
test_data = "武 汉 今 天 下 雨".split()
test_data

['武', '汉', '今', '天', '下', '雨']

In [60]:
def check_mark(training_data):
    """检查标记数目和字是否对应"""
    for pair in training_data:
        if len(pair[1]) == len(pair[0]):
            print("PASS!")
        else:
            print(pair[0], "标记错误")

In [61]:
check_mark(training_data)

PASS!
PASS!
PASS!
PASS!
PASS!
PASS!
PASS!
PASS!
PASS!


### 构建 单词 到 索引 的字典

In [65]:
# 构建 字 到 索引 的字典
word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

# 构建 tag 到 索引 的字典
tag_to_ix = {"B-Location": 0, "I-Location": 1, "E-Location": 2, "O":3, START_TAG: 4, STOP_TAG: 5}
ix_to_tag = { 0:"B-Location", 1:"I-Location", 2:"E-Location", 3:"O"}

In [66]:
# 构建模型
model = BiLSTM_CRF(vocab_size=len(word_to_ix),  # 单词总数
                   tag_to_ix=tag_to_ix,        # tag字典
                   embedding_dim=EMBEDDING_DIM,    # 词嵌入维度
                   hidden_dim=HIDDEN_DIM)       # 隐藏层
# 随机梯度下降
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)  

# Check predictions before training
# 在训练前检查一下预测
with torch.no_grad():
    # 训练数据文本
    precheck_sent = prepare_sequence(training_data[0][0], 
                                     word_to_ix)
    # tags
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print(model(precheck_sent))  # 返回score和标记序列

embedding_dim 20
hidden_dim 64
(tensor(8.7346), [2, 1, 3, 2, 1, 3, 2, 1, 3, 2])


In [67]:
# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(
        100):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        # 梯度归零
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        # 
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(test_data,  # 测试数据
                                     word_to_ix)
    score, tag_seq = model(precheck_sent)
    print("path score:", score)
    print(test_data)
    tag_list = []
    for tag_index in tag_seq:
        tag_list.append(ix_to_tag[tag_index])
    print(tag_list)
    # We got it!

path score: tensor(28.9182)
['武', '汉', '今', '天', '下', '雨']
['B-Location', 'E-Location', 'O', 'O', 'O', 'O']


https://github.com/ThilinaRajapakse/pytorch-transformers-classification

https://github.com/ThilinaRajapakse/pytorch-transformers-classification